In this notebook, I solve the alternative HW1 for Kyle Herkenhoff's Macro Labour class. In this homework, we are asked to solve a simple Bewley model.

In [1]:
using LinearAlgebra, Optim, Plots, SparseArrays

In [2]:
# Params = [beta, gamma]
Params = Vector{Float64}([0.9, 1.0])
# Sizes = [grid_expansion_factor, number_of_states, number_of_actions]
Sizes = Vector{Int64}([2, 2, 101])

3-element Vector{Int64}:
   2
   2
 101

In [3]:
function expanding_grid(S)

    zero_grid = LinRange(0, 1, S[3])

    asset_grid = zeros(S[3], 1)

    a_low = -2.0

    a_high = 10.0

    for i in 1:S[3]

        asset_grid[i] = (zero_grid[i]^S[1]) * (a_high - a_low) + a_low

    end

    return asset_grid

end

expanding_grid (generic function with 1 method)

In [4]:
function states(S)

    states = range(exp(1), exp(2), S[2])

    transition = zeros(S[2], S[2])

    for i in 1:S[2]

        for j in 1:S[2]

            transition[i,j] = 1/S[2]

        end
    
    end

    return states, transition

end

states (generic function with 1 method)

In [5]:
function generate_matrices(S, P)

    value = zeros(S[3], S[2])

    policy = zeros(S[3], S[2])

    return value, policy

end

generate_matrices (generic function with 1 method)

In [6]:
function Bellman_grid_search(S, P, tolerance, max_iterations, q)

    iter = 0

    println("Iterating...")

    state, transition = states(S)

    bond_grid = expanding_grid(S)

    error = tolerance + 1

    Val, Pol = generate_matrices(S, P)

    V_old = similar(Val)

    V_new = similar(Val)

    expected_value = similar(Val)

    util = similar(Val)

    while (error > tolerance) && (max_iterations > 0)

    #for j in 1:S[2]

        for i in 1:S[3]

            consumption = bond_grid[i] .+ state .- (q .* bond_grid')

            c_positive = consumption .> 0

            consumption_positive = consumption .* c_positive

            expected_value = (V_old * transition)

            util =  log.(consumption_positive')

            Val = util .+ P[1] * expected_value

            Val[.!c_positive'] .= -Inf 

            V_new[i,:], vmax_indices = findmax(Val, dims=1)

            index = [vmax_indices[1][1], vmax_indices[2][1]]

            Pol[i,:] = bond_grid[index]

        end

        error = maximum(abs.(V_new - V_old))

        V_old = copy(V_new)

        if isinteger(iter/25) == true
            println("iteration $iter completed with error $error")
        end

        iter += 1

    end

    println("converged in $iter iterations")
    
    #v = Plots.plot(bond_grid, V_new[:,1], title="Value Functions", label="Value 1")

    #Plots.plot!(bond_grid, V_new[:,2], title="Value Functions", label="Value 2")

    #display(v)

    #p = Plots.plot(bond_grid, Pol[:,1], title="Policy Functions", label="Policy 1")

    #Plots.plot!(p, [-2, 10], [-2, 10], color=:black, label="45-degree line")

    #Plots.plot!(bond_grid, Pol[:,2], title="Policy Functions", label="Policy 2")

    #display(p)
    
    return V_new, Pol

end

Bellman_grid_search (generic function with 1 method)

In [8]:
@time begin
Value, Policy = Bellman_grid_search(Sizes, Params, 1e-8, 10000, 1)
end

Iterating...
iteration 0 completed with error 4.233300009900137
iteration 25 completed with error 0.0911435555369593
iteration 50 completed with error 0.006543175799656353
iteration 75 completed with error 0.0004697332739738158
iteration 100 completed with error 3.372205721774435e-5
iteration 125 completed with error 2.420899704702606e-6
iteration 150 completed with error 1.7379590744326379e-7
iteration 175 completed with error 1.2476778721293158e-8
converged in 179 iterations
  0.118838 seconds (308.12 k allocations: 206.539 MiB, 12.63% gc time)


([14.891989705613998 15.97909854727491; 14.892431063530667 15.979310679240708; … ; 17.040084439566844 17.60805039732661; 17.07098416442713 17.635292105059595], [-2.0 -0.2671999999999999; -2.0 -0.2671999999999999; … ; 4.75 8.378800000000002; 4.9312000000000005 8.6032])

In [41]:
function clear_markets(S, P, max_price_iter, quant_tolerance)

    index = zeros(S[3], S[2])

    state, transition = states(S)

    bond_grid = expanding_grid(S)

    q_low = P[1]

    q_high = 1.0

    price_iter = 0

    error = quant_tolerance + 1

    while price_iter < max_price_iter && error > quant_tolerance

        q_update = (q_low + q_high) / 2

        Value, Policy = Bellman_grid_search(Sizes, Params, 1e-8, 10000, q_update)

        M = zeros(S[3], S[2], S[3], S[2])

        for i in 1:S[3]

            for j in 1:S[2]

            index[i,j] = findfirst(isequal(Policy[i,j]), bond_grid)[1]

            end

        end

        for i in 1:S[3]

            for j in 1:S[2]

                for k in 1:S[3]

                    for l in 1:S[2]

                        if Policy[i,j] == bond_grid[k]

                            M[i,j,k,l] = 1/S[2]

                        else 

                            M[i,j,k,l] = 0

                        end

                    end

                end

            end

        end

        O = reshape(M, (S[3]*S[2], S[3]*S[2]))

        number_of_states = S[3]*S[2]

        A = [I - O'; ones(1, number_of_states)]

        b = [zeros(number_of_states); 1]

        x = A\b

        negative_values = x .< 0

        x[negative_values] .= 0

        small_values = abs.(x) .< 1e-6

        x[small_values] .= 0

        x = x ./ sum(x)

        p0 = reshape(Policy, (number_of_states, 1))

        agg = dot(x, p0)

        if agg < 0

            q_low = q_low

            q_high = q_update

        elseif agg > 0

            q_low = q_update

            q_high = q_high

        end
        
        error = abs(q_high - q_low)

        price_iter += 1
        
        println("iteration $price_iter completed with error $error")

    end

    return q_low, q_high
    
end
    

clear_markets (generic function with 1 method)

In [45]:
price = clear_markets(Sizes, Params, 1000, 1e-5)

(0.9771362304687499, 0.9771423339843749)

In [29]:
function check_zero(S,P, q)

    index = zeros(S[3], S[2])

    state, transition = states(S)

    bond_grid = expanding_grid(S)

    Value, Policy = Bellman_grid_search(Sizes, Params, 1e-8, 10000, q)

    M = zeros(S[3], S[2], S[3], S[2])

    for i in 1:S[3]

        for j in 1:S[2]

        index[i,j] = findfirst(isequal(Policy[i,j]), bond_grid)[1]

        end

    end

    for i in 1:S[3]

        for j in 1:S[2]

            for k in 1:S[3]

                for l in 1:S[2]

                    if Policy[i,j] == bond_grid[k]

                        M[i,j,k,l] = 1/S[2]

                    else 

                        M[i,j,k,l] = 0

                    end

                end

            end

        end

    end

    O = reshape(M, (S[3]*S[2], S[3]*S[2]))

    number_of_states = S[3]*S[2]

    A = [I - O'; ones(1, number_of_states)]

    b = [zeros(number_of_states); 1]

    x = A\b

    negative_values = x .< 0

    x[negative_values] .= 0

    small_values = abs.(x) .< 1e-6

    x[small_values] .= 0

    x = x ./ sum(x)

    p0 = reshape(Policy, (number_of_states, 1))

    agg = dot(x, p0)

    return agg

end

check_zero (generic function with 2 methods)

In [43]:
agg_b_0 = check_zero(Sizes, Params, 0.9765625) 

0.007965987046326561

In [44]:
agg_b_1 = check_zero(Sizes, Params, 0.97734375) 

-0.005283879467871538